<a href="https://colab.research.google.com/github/Rohita-G/rohita-gangishetty-700750412/blob/main/ICP9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import files
uploaded = files.upload()

Saving Sentiment (2).csv to Sentiment (2).csv


In [15]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import re

from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('Sentiment (2).csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 2, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)

Epoch 1/2
291/291 - 52s - loss: 0.8232 - accuracy: 0.6460 - 52s/epoch - 178ms/step
Epoch 2/2
291/291 - 46s - loss: 0.6785 - accuracy: 0.7099 - 46s/epoch - 157ms/step
144/144 - 4s - loss: 0.7458 - accuracy: 0.6804 - 4s/epoch - 25ms/step
0.7458383440971375
0.680428147315979
['loss', 'accuracy']


In [16]:
model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
from keras.models import load_model
model = load_model('model.h5')

In [18]:
from keras.models import load_model
# Load the saved model
loaded_model = load_model("model.h5")
# Define a function to preprocess and predict on new text
def predict_sentiment(new_text):
    new_text = new_text.lower()
    new_text = re.sub('[^a-zA-z0-9\s]', '', new_text)
    text_sequence = tokenizer.texts_to_sequences([new_text])
    padded_sequence = pad_sequences(text_sequence, maxlen=X.shape[1])
    sentiment_probabilities = loaded_model.predict(padded_sequence)
    sentiment_label = labelencoder.inverse_transform([sentiment_probabilities.argmax()])[0]
    return sentiment_label
# Example new text for prediction
new_text = "A lot of good things are happening. We are respected again throughout the world, and that's a great thing. @realDonaldTrump"
# Make a prediction
predicted_sentiment = predict_sentiment(new_text)
print("Predicted Sentiment:", predicted_sentiment)

1/1 [==============================] - 0s 316ms/step
Predicted Sentiment: Positive


In [21]:
param_grid = {
    'batch_size': [10, 20, 40],
    'epochs': [1, 2, 3],
    'lstm_out': [128, 196, 256]
}

In [22]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(estimator=createmodel, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy')


In [24]:
from sklearn.base import BaseEstimator

class SentimentModel(BaseEstimator):
    def __init__(self, max_features=2000, embed_dim=128, lstm_out=196):
        self.max_features = max_features
        self.embed_dim = embed_dim
        self.lstm_out = lstm_out

    def fit(self, X, y):
        self.model = createmodel(max_features=self.max_features, embed_dim=self.embed_dim, lstm_out=self.lstm_out)
        self.model.fit(X, y)

    def predict(self, X):
        return self.model.predict(X)

# Create a SentimentModel object
model = SentimentModel()

In [37]:
from sklearn.base import BaseEstimator

class KerasClassifier(BaseEstimator):
    def __init__(self, build_fn, **kwargs):
        self.build_fn = build_fn
        self.kwargs = kwargs

    def fit(self, X, y):
        self.model = self.build_fn()
        self.model.fit(X, y)

    def predict(self, X):
        return self.model.predict(X)

    def get_params(self, deep=True):
        return self.kwargs

In [41]:
from sklearn.metrics import accuracy_score

param_grid = {
    'batch_size': [10, 20, 40],
    'epochs': [1, 2, 3]
}

grid = GridSearchCV(estimator=KerasClassifier(build_fn=model), param_grid=param_grid, scoring=accuracy_score)


In [44]:
grid = GridSearchCV(estimator=KerasClassifier(build_fn=model), param_grid=param_grid)

# Set the scoring metric
grid.scoring = accuracy_score

